In [1]:
import openai
import pandas as pd
import json
from langchain.agents import tool

In [2]:
output_data = pd.read_excel('RPA Pricing Extraction.xlsx')

In [3]:
output_data["Max Discount"]=None
output_data["IBD Bank Name/UPI"]=None
output_data["Min Swipe"]=None
output_data["Bump up"]=None
output_data["VPC(Amazon)"]=None
output_data["SuperCoins(Flipkart)"]=None
output_data.drop('BUP', axis=1, inplace=True)
output_data.drop('VPC', axis=1, inplace=True)
output_data.drop('SuperCoins', axis=1, inplace=True)
output_data

,Market Place,FSN /ASIN,Description,MRP,Current Price,Bank Offers,URL,Payload,API Status,Max Discount,IBD Bank Name/UPI,Min Swipe,Bump up,VPC(Amazon),SuperCoins(Flipkart)
0,Amazon,B0851WN735,Redmi 10000mAh Lithium Polymer Power Bank with...,"₹1,999","1,099\n",1.10% Instant Discount up to INR 1250 on SBI C...,https://www.amazon.in/dp/B0851WN735,NaN,NaN,None,None,None,None,None,None
1,Amazon,B0851WMSDS,Redmi Lithiumpolymer 10000 Mah Fast Charging S...,"₹1,999","1,099\n",1.10% Instant Discount up to INR 1250 on SBI C...,https://www.amazon.in/dp/B0851WMSDS,NaN,NaN,None,None,None,None,None,None
2,Amazon,B08HVL8QN3,"Mi 10000mAH Li-Polymer, Micro-USB and Type C I...","₹2,199","1,199\n",1.10% Instant Discount up to INR 1250 on SBI C...,https://www.amazon.in/dp/B08HVL8QN3,NaN,NaN,None,None,None,None,None,None
3,Amazon,B08HVJCW95,MI 10000mAh 3i Lithium Polymer Power Bank Dual...,"₹2,199","1,199\n",1.10% Instant Discount up to INR 1250 on SBI C...,https://www.amazon.in/dp/B08HVJCW95,NaN,NaN,None,None,None,None,None,None
4,Amazon,B08MC57J31,"MI 10000mAh Lithium Ion, Lithium Polymer Power...","₹2,499","1,599\n",1.10% Instant Discount up to INR 1250 on SBI C...,https://www.amazon.in/dp/B08MC57J31,NaN,NaN,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Amazon,B08Y653FX4,Redmi 164 cm (65 inches) 4K Ultra HD Android S...,"₹74,999","55,999",1.10% Instant Discount up to INR 750 on SBI De...,https://www.amazon.in/dp/B08Y653FX4,NaN,NaN,None,None,None,None,None,None
416,Amazon,B09WN3SRC7,Sony Bravia 164 cm (65 inches) 4K Ultra HD Sma...,"₹1,39,900","72,990",1.Flat INR 2000 Instant Discount on All Banks ...,https://www.amazon.in/dp/B09WN3SRC7,NaN,NaN,None,None,None,None,None,None
417,Amazon,B093LMH4N6,Mi 138.8 cm (55 inches) Q1 Series 4K Smart QLE...,0,"59,999",1.10% Instant Discount up to INR 750 on SBI De...,https://www.amazon.in/dp/B093LMH4N6,NaN,NaN,None,None,None,None,None,None
418,Amazon,B09TBCKPLF,Vu 139 cm (55 inches) The Masterpiece Glo Seri...,"₹80,000","61,990",1.Additional INR 500 Discount on SBI Credit Ca...,https://www.amazon.in/dp/B09TBCKPLF,NaN,NaN,None,None,None,None,None,None


In [11]:
def create_content(price, offers):
    content=f"The Price of the SKU is {price}\n\nOffers:\n\n{offers}\n\nWhich is the best Offer?"
    return content

In [12]:
def process_discount_details(max_discount, min_swipe):
    discount_perc = max_discount/min_swipe*100
    return discount_perc

In [13]:
#Load the LLM
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model='gpt-4-0613', temperature=0)

In [14]:
tools = []

In [15]:

@tool
def best_offer_details(max_discount, bank_name, min_swipe):
    '''Get the discount details like discount amount, bank name and Minimum swipe/pucrhase from the offer which is resulting in maximum discount
    
    Inputs:
        max_discount (integer): The discount amount from the best offer
        bank_name (str): The name of the bank which is offereing the best offer
        min_swipe (int): The minimum swipe or purchase amount from the best offer
    '''
    #print("Discount%", process_discount_details(max_discount, min_swipe))
tools = [best_offer_details]

In [16]:
#Create the prompt

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant which helps in the identification of the best offer. In any offer when not explicitly mentioned assume any percentage discount/cashback as percentage of SKU price. Assume membership if that is one of the condition for any offer. Assume Prime membership wherever applicable & transactions are not Amazon business transactions."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [17]:
#Bind the tool with the LLM
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [18]:
#Create the Agent

from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [19]:
#Create the agent_executor

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [20]:
response = agent_executor.invoke({"input": "The Price of the SKU is 15999\n\nOffers:\n\n1. Flat INR 2100 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n\n6. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer?"})



> Entering new AgentExecutor chain...

Invoking: `best_offer_details` with `{'max_discount': 799.95, 'bank_name': 'Amazon Pay ICICI Bank', 'min_swipe': 15999}`
responded: The SKU price is 15999. 

Looking at the offers:

1. The HDFC Bank Credit Card offer requires a minimum purchase of 17999, which is more than the SKU price. So, this offer is not applicable.

2. The ICICI Bank Credit Card offer also requires a minimum purchase of 17999, which is more than the SKU price. So, this offer is not applicable.

3. The HDFC Bank Debit Card EMI offer requires a minimum purchase of 19999, which is more than the SKU price. So, this offer is not applicable.

4. The HSBC Cashback Card offer gives a 5% discount up to INR 250. So, for a SKU price of 15999, the discount would be 5% of 15999, which is 799.95. But the maximum discount is capped at INR 250. So, the discount for this offer is INR 250.

5. The Amazon Pay ICICI Bank credit card offer gives 5% back for Prime members. So, the discount woul

In [21]:
response['intermediate_steps'][0][0].tool_input

{'max_discount': 799.95,
 'bank_name': 'Amazon Pay ICICI Bank',
 'min_swipe': 15999}